=Turing.jl=
===========

**Author:** Tor Erlend Fjelde



## Before we begin



Make sure you're in the correct directory



In [ ]:
pwd()

Then run something like (depending on which OS you are on)

or if you're already in a REPL, do



In [ ]:
]activate .

to activate the project

And just to check that you're in the correct one



In [ ]:
]status

Download and install dependencies



In [ ]:
]instantiate

And finally, do



In [ ]:
using GeiloWinterSchool2023Part2

to get some functionality I've implemented for the occasion



## The story of a little Norwegian boy



There once was a little Norwegian boy

![img](.notes/attachments/A_litle_Norwegian_boy/2023-01-18_14-49-24_471337_3317365246956_1262712540_o.jpg)

When this little boy was 20 years old, he was working as a parking guard near Preikestolen/Pulpit rock

![img](.notes/attachments/A_litle_Norwegian_boy/2023-01-18_14-57-08_Preikestolen-plateau-Go-Fjords-Bob-Engelsen-P1026771_kljg5o.jpeg)

One day it was raining and there was nobody hiking, and so there was no cars in sight for the little boy to point

<div class="fragment (appear)">

When his boss wasn't looking, the little 20 year-old boy had an amazing idea

> Maybe I can use this method of Mr. Bayes I learned a bit about yesteday to model football / Premier League?#+HTML: </div>

The little boy got very excited and started looking for stuff on the big interwebs

The little boy came across this

![img](.notes/attachments/A_litle_Norwegian_boy/2023-01-18_14-46-02_Screenshot_20230118_144454.png)

And got <u>very</u> excited

But at the time, the little boy knew next to <u>nothing</u> about programming

The little boy couldn't write the code to do the inference

Whence the little boy became a <u>sad</u> little boy :(

But time heals all wounds, and at some point the little boy learned Python

And in Python, the boy found the *probabilistic programming language* `pymc3`

<div class="fragment (appear)">

> Maybe I can use `pymc3` to perform inference in that football / Premier League model?And so the sad boy once more became an <u>excited</u> little boy :)

</div>

But there was a problem

The boy wanted to write a for-loop in his model, but the model didn't want it to be so and complained!

The boy got frustrated and gave up, once more becoming a <u>sad</u> little boy :(

<div class="small-text">

The boy should have known that the computational backend `theano` that was used by `pymc3` at the time couldn't handle a for-loop, and instead he should have used `scan`. But the boy was only 20-something years old; he didn't know.

</div>

Some years later the boy discovers a programming language called <u>Julia</u>

<div class="fragment (appear)">

Julia makes a few promises

1.  It's fast. Like *really* fast.
2.  It's interactive; doesn't require full compilation for you to play with it.
3.  You don't have to specify types everywhere.

</div>

<div class="fragment (appear)">

The boy thinks

> Wait, but this sounds like Python but the only difference is that&#x2026;I CAN WRITE FOR-LOOPS WITHOUT FEELING BAD ABOUT IT?!Yes, yes he could

And 3.5 years later, he's still writing for-loops. Well, sort of.

</div>



## Why Turing.jl?



Duh, you should use Turing.jl <u>so you get to use Julia</u>

<div class="fragment (appear)">

But even in Julia, other PPLS exist

But Turing.jl is very similar to Julia in "philosophy":

-   Flexiblility
-   Ease-of-use
-   Speed (potentially with a bit of effort)

So it's a pretty good candidate

</div>



## Running example



We'll work with an outbreak of influenza A (H1N1) in 1978 at a British boarding school

-   763 male students -> 512 of which became ill
-   Reported that one infected boy started the epidemic
-   Observations are number of boys in bed over 14 days

Data are freely available in the R package `outbreaks`, maintained as part of the [R Epidemics Consortium](http://www.repidemicsconsortium.org/)

<div class="fragment (appear)">

Data + part of the analysis is *heavily* inspired by [https://mc-stan.org/users/documentation/case-studies/boarding_school_case_study.html](https://mc-stan.org/users/documentation/case-studies/boarding_school_case_study.html)

Stan definitively beats Turing.jl when it comes to great write-ups like these

</div>



### Loading into Julia



In [ ]:
# Load the dataframe.
using Dates
using DataFrames, CSV

N = 763
data = DataFrame(CSV.File(joinpath("data", "influenza_england_1978_school.csv")));
print(data)

Notice that each of the columns have associated types

Let's visualize the samples:



In [ ]:
using StatsPlots

In [ ]:
# StatsPlots.jl provides this convenient macro `@df` for plotting a `DataFrame`.
@df data scatter(:date, :in_bed, label=nothing, ylabel="Number of students in bed")

## Differential equations



Suppose we have some function $f$ which describes how a state $x$ evolves wrt. $t$

\begin{equation*}
\frac{\mathrm{d} x}{\mathrm{d} t} = f(x, t)
\end{equation*}

which we then need to integrate to obtain the actual state at some time $t$

\begin{equation*}
x(t) = \int_{0}^{t} \frac{\mathrm{d} x}{\mathrm{d} t} \mathrm{d} t = \int_{0}^{t} f(x, t) \mathrm{d} t
\end{equation*}

In many interesting scenarios numerical methods are required to obtain $x(t)$



### In Julia



Everything related to differential equations is provided by [`DifferentialEquations.jl`](https://docs.sciml.ai/DiffEqDocs/stable/)

And I really do mean [*everything*](https://docs.sciml.ai/DiffEqDocs/stable/)

<div class="side-by-side">

![img](.notes/attachments/Differential_equations/2023-01-19_19-48-23_Screenshot_20230119_194737.png)

![img](.notes/attachments/Differential_equations/2023-01-19_19-48-41_Screenshot_20230119_194838.png)

</div>



### Example: SIR model



One particular example of an (ordinary) differential equation that you might have seen recently is the **SIR model** used in epidemiology

![img](.notes/attachments/Differential_equations/2023-01-19_19-56-00_sir_illu.png "[https://covid19.uclaml.org/model.html>](https://covid19.uclaml.org/model.html>)(2023-01-19)")

The temporal dynamics of the sizes of each of the compartments are governed by the following system of ODEs:

\begin{equation*}
\begin{split}
  \frac{\mathrm{d} S}{\mathrm{d} t} &= - \beta S \frac{I}{N} \\
  \frac{\mathrm{d} I}{\mathrm{d} t} &= \beta S \frac{I}{N} - \gamma I \\
  \frac{\mathrm{d} R}{\mathrm{d} t} &= \gamma I
\end{split}
\end{equation*}

where

-   $S(t)$ is the number of people susceptible to becoming infected,
-   $I(t)$ is the number of people currently infected,
-   $R(t)$ is the number of recovered people,
-   $β$ is the constant rate of infectious contact between people,
-   $\gamma$ the constant recovery rate of infected individuals

Converting this ODE into code is just



In [ ]:
using DifferentialEquations

function SIR!(
    du,  # buffer for the updated differential equation
    u,   # current state
    p,   # parameters
    t    # current time
)
    N = 763  # population
    S, I, R = u
    β, γ = p

    du[1] = dS = -β * I * S / N
    du[2] = dI = β * I * S / N - γ * I
    du[3] = dR = γ * I
end

Not too bad!

Initial conditions are then

\begin{equation*}
\begin{split}
  S(0) &= N - 1 \\
  I(0) &= 1 \\
  R(0) &= 0
\end{split}
\end{equation*}

and we want to integrate from $t = 0$ to $t = 14$



In [ ]:
# Include 0 because that's the initial condition before any observations.
tspan = (0.0, 14.0)

# Initial conditions are:
#   S(0) = N - 1; I(0) = 1; R(0) = 0
u0 = [N - 1, 1, 0.0]

Now we just need to define the overall problem and we can solve:



In [ ]:
# Just to check that everything works, we'll just use some "totally random" values for β and γ:
problem_sir = let β = 2.0, γ = 0.6
    ODEProblem(SIR!, u0, tspan, (β, γ))
end

Aaaand



In [ ]:
sol = solve(problem_sir)

We didn't specify a solver

DifferentialEquations.jl uses `AutoTsit5(Rosenbrock32())` by default 

Which is a composition between

-   `Tsit5` (4th order Runge-Kutta), and
-   `Rosenbrock32` (3rd order stiff solver)

with automatic switching between the two

`AutoTsit5(Rosenbrock32())` covers many use-cases well, but see

-   [https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/](https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/)
-   [https://www.stochasticlifestyle.com/comparison-differential-equation-solver-suites-matlab-r-julia-python-c-fortran/](https://www.stochasticlifestyle.com/comparison-differential-equation-solver-suites-matlab-r-julia-python-c-fortran/)

for more info on choosing a solver

This is the resulting solution



In [ ]:
plot(
    sol,
    linewidth=2, xaxis="Time in days", label=["Suspectible" "Infected" "Recovered"],
    alpha=0.5, size=(500, 300)
)
scatter!(1:14, data.in_bed, label="Data", color="black")

This doesn't really match the data though; let's do better

Approach #1: find optimal values of $\beta$ and $\gamma$ by minimizing some loss, e.g. sum-of-squares

\begin{equation*}
\ell(\beta, \gamma) = \sum_{i = 1}^{14} \bigg( F(u_0, t_i;\ \beta, \gamma) - y_i \bigg)^2
\end{equation*}

where $\big( y_i \big)_{i = 1}^{14}$ are the observations, $F$ is the integrated system

<div class="fragment (appear)">

First we define the loss



In [ ]:
# Define the loss function.
function loss_sir(problem_orig, p)
    # `remake` just, well, remakes the `problem` with `p` replaced.
    problem = remake(problem_orig, p=p)
    # To ensure we get solutions _exactly_ at the timesteps of interest,
    # i.e. every day we have observations, we use `saveat=1` to tell `solve`
    # to save at every timestep (which is one day).
    sol = solve(problem, saveat=1)
    # Extract the 2nd state, the (I)infected, for the dates with observations.
    sol_for_observed = sol[2, 2:15]
    # Compute the sum-of-squares of the infected vs. data.
    sum(abs2.(sol_for_observed - data.in_bed))
end

</div>

And the go-to for optimization in Julia is [Optim.jl](https://julianlsolvers.github.io/Optim.jl/stable/)



In [ ]:
using Optim
# An alternative to writing `y -> f(x, y)` is `Base.Fix1(f, x)` which
# avoids potential performance issues with global variables (as our `problem` here).
opt = optimize(
    p -> loss_sir(problem_sir, p), # function to minimize
    [0, 0],                # lower bounds on variables
    [Inf, Inf],            # upper bounds on variables
    [2.0, 0.5],            # initial values
    Fminbox(NelderMead())  # optimization alg
)

We can extract the minimizers of the loss



In [ ]:
β, λ = Optim.minimizer(opt)
β, λ

In [ ]:
# Solve for the obtained parameters.
problem = remake(problem_sir, p=(β, λ))
sol = solve(problem_sir)

# Plot the solution.
plot(sol, linewidth=2, xaxis="Time in days", label=["Suspectible" "Infected" "Recovered"], alpha=0.5)
# And the data.
scatter!(1:14, data.in_bed, label="Data", color="black")

That's better than our *totally* "random" guess from earlier!



### Example: SEIR model



Adding another compartment to our SIR model: the <u>(E)xposed</u> state

\begin{equation*}
\begin{split}
  \frac{\mathrm{d} S}{\mathrm{d} t} &= - \beta S \frac{I}{N} \\
  \frac{\mathrm{d} {\color{blue} E}}{\mathrm{d} t} &= \beta S \frac{I}{N} - {\color{orange} \sigma} {\color{blue} E} \\
  \frac{\mathrm{d} I}{\mathrm{d} t} &= {\color{orange} \sigma} {\color{blue} E} - \gamma I \\
  \frac{\mathrm{d} R}{\mathrm{d} t} &= \gamma I
\end{split}
\end{equation*}

where we've added a new parameter ${\color{orange} \sigma}$ describing the fraction of people who develop observable symptoms in this time



### TASK Solve the SEIR model using Julia



In [ ]:
function SEIR!(
    du,  # buffer for the updated differential equation
    u,   # current state
    p,   # parameters
    t    # current time
)
    N = 763  # population

    S, E, I, R = u  # have ourselves an additional state!
    β, γ, σ = p     # and an additional parameter!

    # TODO: Implement yah fool!
    du[1] = nothing
    du[2] = nothing
    du[3] = nothing
    du[4] = nothing
end

**BONUS:** Use `Optim.jl` to find minimizers of sum-of-squares



### SOLUTION Solve the SEIR model using Julia



In [ ]:
function SEIR!(
    du,  # buffer for the updated differential equation
    u,   # current state
    p,   # parameters
    t    # current time
)
    N = 763  # population
    S, E, I, R = u  # have ourselves an additional state!
    β, γ, σ = p     # and an additional parameter!

    # Might as well cache these computations.
    βSI = β * S * I / N
    σE = σ * E
    γI = γ * I

    du[1] = -βSI
    du[2] = βSI - σE
    du[3] = σE - γI
    du[4] = γI
end

In [ ]:
problem_seir = let u0 = [N - 1, 0, 1, 0], β = 2.0, γ = 0.6, σ = 0.8
    ODEProblem(SEIR!, u0, tspan, (β, γ, σ))
end

In [ ]:
sol_seir = solve(problem_seir, saveat=1)

In [ ]:
plot(sol_seir, linewidth=2, xaxis="Time in days", label=["Suspectible" "Exposed" "Infected" "Recovered"], alpha=0.5)
scatter!(1:14, data.in_bed, label="Data")

Don't look so good. Let's try Optim.jl again.



In [ ]:
function loss_seir(problem, p)
    problem = remake(problem, p=p)
    sol = solve(problem, saveat=1)
    # NOTE: 3rd state is now the (I)nfectious compartment!!!
    sol_for_observed = sol[3, 2:15]
    return sum(abs2.(sol_for_observed - data.in_bed))
end

In [ ]:
opt = optimize(Base.Fix1(loss_seir, problem_seir), [0, 0, 0], [Inf, Inf, Inf], [2.0, 0.5, 0.9], Fminbox(NelderMead()))

In [ ]:
β, γ, σ = Optim.minimizer(opt)

In [ ]:
sol_seir = solve(remake(problem_seir, p=(β, γ, σ)), saveat=1)
plot(sol_seir, linewidth=2, xaxis="Time in days", label=["Suspectible" "Exposed" "Infected" "Recovered"], alpha=0.5)
scatter!(1:14, data.in_bed, label="Data", color="black")

> But&#x2026;but these are <u>point estimates</u>! What about distributions? WHAT ABOUT UNCERTAINTY?!No, no that's fair.

Let's do some Bayesian inference then.

BUT FIRST!



### Making our future selves less annoyed



It's annoying to have all these different loss-functions for *both* `SIR!` and `SEIR!`

<div class="fragment (appear)">



In [ ]:
# Abstract type which we can use to dispatch on.
abstract type AbstractEpidemicProblem end

struct SIRProblem{P} <: AbstractEpidemicProblem
    problem::P
    N::Int
end

function SIRProblem(N::Int; u0 = [N - 1, 1, 0.], tspan = (0, 14), p = [2.0, 0.6])
    return SIRProblem(ODEProblem(SIR!, u0, tspan, p), N)
end

Then we can just construct the problem as



In [ ]:
sir = SIRProblem(N);

</div>

And to make it a bit easier to work with, we add some utility functions



In [ ]:
# General.
parameters(prob::AbstractEpidemicProblem) = prob.problem.p
initial_state(prob::AbstractEpidemicProblem) = prob.problem.u0
population(prob::AbstractEpidemicProblem) = prob.N

# Specializations.
susceptible(::SIRProblem, u::AbstractMatrix) = u[1, :]
infected(::SIRProblem, u::AbstractMatrix) = u[2, :]
recovered(::SIRProblem, u::AbstractMatrix) = u[3, :]

So that once we've solved the problem, we can easily extract the compartment we want, e.g.



In [ ]:
sol = solve(sir.problem, saveat=1)
infected(sir, sol)

### TASK Implement `SEIRProblem`



In [ ]:
struct SEIRProblem <: AbstractEpidemicProblem
    # ...
end

function SEIRProblem end

susceptible
exposed
infected
recovered

### SOLUTION Implement `SEIRProblem`



In [ ]:
struct SEIRProblem{P} <: AbstractEpidemicProblem
    problem::P
    N::Int
end

function SEIRProblem(N::Int; u0 = [N - 1, 0, 1, 0.], tspan = (0, 14), p = [4.5, 0.45, 0.8])
    return SEIRProblem(ODEProblem(SEIR!, u0, tspan, p), N)
end

susceptible(::SEIRProblem, u::AbstractMatrix) = u[1, :]
exposed(::SEIRProblem, u::AbstractMatrix) = u[2, :]
infected(::SEIRProblem, u::AbstractMatrix) = u[3, :]
recovered(::SEIRProblem, u::AbstractMatrix) = u[4, :]

Now, given a `problem` and a `sol`, we can query the `sol` for the `infected` state without explicit handling of which `problem` we're working with



In [ ]:
seir = SEIRProblem(N);
sol = solve(seir.problem, saveat=1)
infected(seir, sol)

### Same `loss` for both!



In [ ]:
function loss(problem_wrapper::AbstractEpidemicProblem, p)
    # NOTE: Extract the `problem` from `problem_wrapper`.
    problem = remake(problem_wrapper.problem, p=p)
    sol = solve(problem, saveat=1)
    # NOTE: Now this is completely general!
    sol_for_observed = infected(problem_wrapper, sol)[2:end]
    return sum(abs2.(sol_for_observed - data.in_bed))
end

Now we can call the same `loss` for both `SIR` and `SEIR`



In [ ]:
loss(SIRProblem(N), [2.0, 0.6])

In [ ]:
loss(SEIRProblem(N), [2.0, 0.6, 0.8])

## Bayesian inference



First off



In [ ]:
using Turing

This dataset really doesn't have too many observations



In [ ]:
nrow(data)

So reporting a single number for parameters is maybe being a *bit* too confident

We'll use the following model

\begin{equation*}
\begin{split}
  \beta &\sim \mathcal{N}_{ + }(2, 1) \\
  \gamma &\sim \mathcal{N}_{ + }(0.4, 0.5) \\
  \phi^{-1} &\sim \mathrm{Exponential}(1/5) \\
   y_i &\sim \mathrm{NegativeBinomial2}\big(F(u_0, t_i;\ \beta, \gamma), \phi \big)
\end{split}
\end{equation*}

where 

-   $\big( y_i \big)_{i = 1}^{14}$ are the observations,
-   $F$ is the integrated system, and
-   $\phi$ is the over-dispersion parameter.



In [ ]:
plot(
    plot(truncated(Normal(2, 1); lower=0), label=nothing, title="β"),
    plot(truncated(Normal(0.4, 0.5); lower=0), label=nothing, title="γ"),
    plot(Exponential(1/5), label=nothing, title="ϕ⁻¹"),
    layout=(3, 1)
)

A `NegativeBinomial(r, p)` represents the number of trials to achieve $r$ successes, where each trial has a probability $p$ of success

A `NegativeBinomial2(μ, ϕ)` is the same, but parameterized using the mean $μ$ and *dispersion* $\phi$



In [ ]:
# `NegativeBinomial` already exists, so let's just make an alternative constructor instead.
function NegativeBinomial2(μ, ϕ)
    p = 1/(1 + μ/ϕ)
    r = ϕ
    return NegativeBinomial(r, p)
end

In [ ]:
# Let's just make sure we didn't do something stupid.
μ = 2; ϕ = 3;
dist = NegativeBinomial2(μ, ϕ)
# Source: https://mc-stan.org/docs/2_20/functions-reference/nbalt.html
mean(dist) ≈ μ && var(dist) ≈ μ + μ^2 / ϕ

Can be considered a generalization of `Poisson`



In [ ]:
μ = 2.0
anim = @animate for ϕ ∈ [0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 25.0, 100.0]
    p = plot(size=(500, 300))
    plot!(p, Poisson(μ); label="Poisson($μ)")
    plot!(p, NegativeBinomial2(μ, ϕ), label="NegativeBinomial2($μ, $ϕ)")
    xlims!(0, 20); ylims!(0, 0.35);
    p
end
gif(anim, "negative_binomial.gif", fps=2);

![img](./negative_binomial.gif)



In [ ]:
@model function sir_model(
    num_days;                                  # Number of days to model
    tspan = (0.0, float(num_days)),            # Timespan to model
    u0 = [N - 1, 1, 0.0],                      # Initial state
    p0 = [2.0, 0.6],                           # Placeholder parameters
    problem = ODEProblem(SIR!, u0, tspan, p0)  # Create problem once so we can `remake`.
)
    β ~ truncated(Normal(2, 1); lower=0)
    γ ~ truncated(Normal(0.4, 0.5); lower=0)
    ϕ⁻¹ ~ Exponential(1/5)
    ϕ = inv(ϕ⁻¹)

    problem_new = remake(problem, p=[β, γ])  # Replace parameters `p`.
    sol = solve(problem_new, saveat=1)       # Solve!

    sol_for_observed = sol[2, 2:num_days + 1]  # Timesteps we have observations for.
    in_bed = Vector{Int}(undef, num_days)
    for i = 1:length(sol_for_observed)
        # Add a small constant to `sol_for_observed` to make things more stable.
        in_bed[i] ~ NegativeBinomial2(sol_for_observed[i] + 1e-5, ϕ)
    end

    # Some quantities we might be interested in.
    return (R0 = β / γ, recovery_time = 1 / γ, infected = sol_for_observed)
end

Let's break it down



In [ ]:
β ~ truncated(Normal(2, 1); lower=0)
γ ~ truncated(Normal(0.4, 0.5); lower=0)
ϕ⁻¹ ~ Exponential(1/5)
ϕ = inv(ϕ⁻¹)

defines our prior

`truncated` is just a way of restricting the domain of the distribution you pass it



In [ ]:
problem_new = remake(problem, p=[β, γ])  # Replace parameters `p`.
sol = solve(problem_new, saveat=1)       # Solve!

We then remake the problem, now with the parameters `[β, γ]` sampled above

`saveat = 1` gets us the solution at the timesteps `[0, 1, 2, ..., 14]`

Then we extract the timesteps we have observations for



In [ ]:
sol_for_observed = sol[2, 2:num_days + 1]  # Timesteps we have observations for.

and define what's going to be a likelihood (once we add observations)



In [ ]:
in_bed = Vector{Int}(undef, num_days)
for i = 1:length(sol_for_observed)
    # Add a small constant to `sol_for_observed` to make things more stable.
    in_bed[i] ~ NegativeBinomial2(sol_for_observed[i] + 1e-5, ϕ)
end

Finally we return some values that might be of interest to



In [ ]:
# Some quantities we might be interested in.
return (R0 = β / γ, recovery_time = 1 / γ, infected = sol_for_observed)

This is useful for a post-sampling diagnostics, debugging, etc.



In [ ]:
model = sir_model(length(data.in_bed))

The model is just another function, so we can call it to check that it works

<div class="fragment (appear)">



In [ ]:
model().infected

Hey, it does!

</div>



### Is the prior reasonable?



Before we do any inference, we should check if the prior is reasonable

From domain knowledge we know that (for influenza at least)

-   $R_0$ is typically between 1 and 2
-   `recovery_time` ($1 / \gamma$) is usually ~1 week

<div class="fragment (appear)">

We want to make sure that your prior belief reflects this knowledge while still being flexible enough to accommodate the observations

</div>

To check this we'll just simulate some draws from our prior model, i.e. the model *without* conditioning on `in_bed`

There are two ways to sample form the prior

<div class="fragment (appear)">



In [ ]:
# 1. By just calling the `model`, which returns a `NamedTuple` containing the quantities of interest
print(model())

</div>

<div class="fragment (appear)">

Or by just calling `sample` using `Prior`



In [ ]:
# Sample from prior.
chain_prior = sample(model, Prior(), 10_000);

</div>



Let's have a look at the prior predictive



In [ ]:
p = plot(legend=false, size=(600, 300))
plot_trajectories!(p, group(chain_prior, :in_bed); n = 1000)
hline!([N], color="red")

For certain values we get number of infected *larger* than the actual population

But this is includes the randomness from `NegativeBinomial2` likelihood

Maybe more useful to inspect the (I)nfected state from the ODE solution?

We can also look at the `generated_quantities`, i.e. the values from the `return` statement in our model

Our `return` looked like this



In [ ]:
# Some quantities we might be interested in.
return (R0 = β / γ, recovery_time = 1 / γ, infected = sol_for_observed)

and so `generated_quantities` (conditioned on `chain_prior`) gives us



In [ ]:
quantities_prior = generated_quantities(
    model,
    MCMCChains.get_sections(chain_prior, :parameters)
)
print(quantities_prior[1])

We can convert it into a `Chains` using a utility function of mine



In [ ]:
# Convert to `Chains`.
chain_quantities_prior = to_chains(quantities_prior);

# Plot.
p = plot(legend=false, size=(600, 300))
plot_trajectories!(p, group(chain_quantities_prior, :infected); n = 1000)
hline!([N], color="red")

<div class="x-small-text">

**NOTE:** `to_chains` is not part of "official" Turing.jl because the `return` can contain *whatever* you want, and so it's not always possible to convert into a `Chains`

</div>

And the quantiles for the trajectories



In [ ]:
p = plot(legend=false, size=(600, 300))
plot_trajectory_quantiles!(p, group(chain_quantities_prior, :infected))
hline!(p, [N], color="red")

In [ ]:
DataFrame(quantile(chain_quantities_prior[:, [:R0, :recovery_time], :]))

Compare to our prior knowledge of $R_0 \in [1, 2]$ and $(1/\gamma) \approx 1$ for influenza

Do we really need probability mass on $R_0 \ge 10$?



### TASK What's wrong with the current prior?



<div class="side-by-side">

<div style="margin: auto;">

The SIR model

\begin{equation*}
\begin{split}
  \frac{\mathrm{d} S}{\mathrm{d} t} &= - \beta S \frac{I}{N} \\
  \frac{\mathrm{d} I}{\mathrm{d} t} &= \beta S \frac{I}{N} - \gamma I \\
  \frac{\mathrm{d} R}{\mathrm{d} t} &= \gamma I
\end{split}
\end{equation*}

</div>

<div>

And here's the current priors

<div class="x-small-text">



In [ ]:
plot(
    plot(truncated(Normal(2, 1); lower=0), label=nothing, title="β"),
    plot(truncated(Normal(0.4, 0.5); lower=0), label=nothing, title="γ"),
    plot(Exponential(1/5), label=nothing, title="ϕ⁻¹"),
    layout=(3, 1)
)

</div>

</div>

</div>



### SOLUTION Recovery time shouldn't be several years



We mentioned that `recovery_time`, which is expressed as $1 / \gamma$, is ~1 week

We're clearly putting high probability on regions near 0, i.e. *long* recovery times



In [ ]:
plot(truncated(Normal(0.4, 0.5); lower=0), label=nothing, title="γ", size=(500, 300))

<u>Should probably be putting less probability mass near 0</u>



### SOLUTION ${\color{red} \gamma}$ should not be larger than 1



\begin{equation*}
\begin{split}
  \frac{\mathrm{d} S}{\mathrm{d} t} &= - \beta S \frac{I}{N} \\
  \frac{\mathrm{d} I}{\mathrm{d} t} &= \beta S \frac{I}{N} - {\color{red} \gamma I} \\
  \frac{\mathrm{d} R}{\mathrm{d} t} &= {\color{red} \gamma I}
\end{split}
\end{equation*}

If ${\color{red} \gamma} > 1$ ⟹ (R)ecovered increase by *more* than the (I)nfected

⟹ <u>healthy people are recovering</u>

Now, I'm no epidemiologist, but that doesn't seem right

Maybe something like



In [ ]:
plot(Beta(2, 5), label="new", size=(500, 300))
plot!(truncated(Normal(0.4, 0.5); lower=0), label="old", color="red")

-   [X] Bounded at 1
-   [X] Allows smaller values (i.e. longer recovery time) but rapidly decreases near zero



### SOLUTION What if ${\color{red} \beta} > N$?



Then for $t = 0$ we have

\begin{equation*}
\frac{\mathrm{d} S}{\mathrm{d} t} \bigg|_{t = 0} = - {\color{red} \beta} S \frac{I}{N} > - N (N - 1) \frac{1}{N} = - (N - 1)
\end{equation*}

i.e. we *immediately* infect everyone on the very first time-step

Also doesn't seem very realistic

*But* under our current prior does this matter?



In [ ]:
# ℙ(β > N) = 1 - ℙ(β ≤ N)
1 - cdf(truncated(Normal(2, 1); lower=0), N)

Better yet



In [ ]:
quantile(truncated(Normal(2, 1); lower=0), 0.95)

i.e. 95% of the probability mass falls below ~3.65

⟹ <u>Current prior for $\beta$ seems fine (✓)</u>

Before we change the prior, let's also make it a bit easier to change the prior using `@submodel`

<div class="fragment (appear)">

`@submodel` allows you call models within models, e.g.



In [ ]:
@model function A()
    x_hidden_from_B ~ Normal()
    x = x_hidden_from_B + 100
    return x
end

@model function B()
    @submodel x = A()
    y ~ Normal(x, 1)

    return (; x, y)
end

</div>

<div class="fragment (appear)">



In [ ]:
# So if we call `B` we only see `x` and `y`
println(B()())

</div>

<div class="fragment (appear)">



In [ ]:
# While if we sample from `B` we get the latent variables
println(rand(B()))

</div>

To avoid clashes of variable-names, we can specify a `prefix`



In [ ]:
@model A() = (x ~ Normal(); return x + 100)

@model function B()
    # Given it a prefix to use for the variables in `A`.
    @submodel prefix=:inner x_inner = A()
    x ~ Normal(x_inner, 1)

    return (; x_inner, x)
end

In [ ]:
print(rand(B()))

`@submodel` is useful as it allows you to:

1.  Easy to swap out certain parts of your model.
2.  Can re-use models across projects and packages.

When working on larger projects, this really shines

Equipped with `@submodel` we can replace



In [ ]:
β ~ truncated(Normal(2, 1); lower=0)
γ ~ truncated(Normal(0.4, 0.5); lower=0)

with



In [ ]:
@submodel p = prior(problem_wrapper)

<div class="fragment (appear)">

where `prior` can be something like



In [ ]:
@model function prior_original(problem_wrapper::SIRProblem)
    β ~ truncated(Normal(2, 1); lower=0)
    γ ~ truncated(Normal(0.4, 0.5); lower=0)

    return [β, γ]
end

@model function prior_improved(problem_wrapper::SIRProblem)
    # NOTE: Should probably also lower mean for `β` since
    # more probability mass on small `γ` ⟹ `R0 =  β / γ` grows.
    β ~ truncated(Normal(1, 1); lower=0)
    # NOTE: New prior for `γ`.
    γ ~ Beta(2, 5)

    return [β, γ]
end

</div>



In [ ]:
@model function epidemic_model(
    problem_wrapper::AbstractEpidemicProblem,
    prior  # NOTE: now we just pass the prior as an argument
)
    # NOTE: And use `@submodel` to embed the `prior` in our model.
    @submodel p = prior(problem_wrapper)

    ϕ⁻¹ ~ Exponential(1/5)
    ϕ = inv(ϕ⁻¹)

    problem_new = remake(problem_wrapper.problem, p=p)  # Replace parameters `p`.
    sol = solve(problem_new, saveat=1)                  # Solve!

    # Extract the `infected`.
    sol_for_observed = infected(problem_wrapper, sol)[2:end]

    # NOTE: `arraydist` is faster for larger dimensional problems,
    # and it does not require explicit allocation of the vector.
    in_bed ~ arraydist(NegativeBinomial2.(sol_for_observed .+ 1e-5, ϕ))

    β, γ = p[1:2]
    return (R0 = β / γ, recovery_time = 1 / γ, infected = sol_for_observed)
end

<div class="x-small-text">

Another neat trick is to return early if integration fail

</div>



In [ ]:
@model function epidemic_model(
    problem_wrapper::AbstractEpidemicProblem,
    prior  # now we just pass the prior as an argument
)
    # And use `@submodel` to embed the `prior` in our model.
    @submodel p = prior(problem_wrapper)

    ϕ⁻¹ ~ Exponential(1/5)
    ϕ = inv(ϕ⁻¹)

    problem_new = remake(problem_wrapper.problem, p=p)  # Replace parameters `p`.
    sol = solve(problem_new, saveat=1)                  # Solve!

    # NOTE: Return early if integration failed.
    if !issuccess(sol)
        Turing.@addlogprob! -Inf  # NOTE: Causes automatic rejection.
        return nothing
    end

    # Extract the `infected`.
    sol_for_observed = infected(problem_wrapper, sol)[2:end]

    # `arraydist` is faster for larger dimensional problems,
    # and it does not require explicit allocation of the vector.
    in_bed ~ arraydist(NegativeBinomial2.(sol_for_observed .+ 1e-5, ϕ))

    β, γ = p[1:2]
    return (R0 = β / γ, recovery_time = 1 / γ, infected = sol_for_observed)
end

Equipped with this we can now easily construct *two* models using different priors



In [ ]:
sir = SIRProblem(N);
model_original = epidemic_model(sir, prior_original);
model_improved = epidemic_model(sir, prior_improved);

but using the same underlying `epidemic_model`



In [ ]:
chain_prior_original = sample(model_original, Prior(), 10_000; progress=false);
chain_prior_improved = sample(model_improved, Prior(), 10_000; progress=false);

Let's compare the resulting priors over some of the quantities of interest

Let's compare the `generated_quantities`, e.g. $R_0$

<div class="small-text">



In [ ]:
chain_quantities_original = to_chains(
    generated_quantities(
        model_original,
        MCMCChains.get_sections(chain_prior_original, :parameters)
    );
);

chain_quantities_improved = to_chains(
    generated_quantities(
        model_improved,
        MCMCChains.get_sections(chain_prior_improved, :parameters)
    );
);

In [ ]:
p = plot(; legend=false, size=(500, 200))
plot_trajectories!(p, group(chain_quantities_original, :infected); n = 100, trajectory_color="red")
plot_trajectories!(p, group(chain_quantities_improved, :infected); n = 100, trajectory_color="blue")
hline!([N], color="red", linestyle=:dash)

</div>

<div class="small-text">



In [ ]:
plt1 = plot(legend=false)
plot_trajectory_quantiles!(plt1, group(chain_quantities_original, :infected))
hline!(plt1, [N], color="red", linestyle=:dash)

plt2 = plot(legend=false)
plot_trajectory_quantiles!(plt2, group(chain_quantities_improved, :infected))
hline!(plt2, [N], color="red", linestyle=:dash)

plot(plt1, plt2, layout=(2, 1))

</div>

This makes sense: if half of the population is immediately infected ⟹ number of infected tapers wrt. time as they recover

For `model_improved` we then have



In [ ]:
DataFrame(quantile(chain_quantities_improved[:, [:R0, :recovery_time], :]))

Compare to `model_original`



In [ ]:
DataFrame(quantile(chain_quantities_original[:, [:R0, :recovery_time], :]))

### TASK Make `epidemic_model` work for `SEIRProblem`



1.  [ ] Implement a prior which also includes $\sigma$ and execute
    `epidemic_model` with it
2.  [ ] Can we make a better prior for $\sigma$? Do we even need one?



In [ ]:
@model function prior_original(problem_wrapper::SEIRProblem)
    # TODO: Implement
end

### SOLUTION



In [ ]:
@model function prior_original(problem_wrapper::SEIRProblem)
    β ~ truncated(Normal(2, 1); lower=0)
    γ ~ truncated(Normal(0.4, 0.5); lower=0)
    σ ~ truncated(Normal(0.8, 0.5); lower=0)

    return [β, γ, σ]
end

In [ ]:
model_seir = epidemic_model(SEIRProblem(N), prior_original)
print(model_seir())

### WARNING Consult with domain experts



This guy should <u>not</u> be the one setting your priors!

![img](.notes/attachments/A_litle_Norwegian_boy/2023-01-18_14-49-24_471337_3317365246956_1262712540_o.jpg)

Get an actual scientist to do that&#x2026;



### Condition



Now let's actually involve the data

<div class="fragment (appear)">

We can condition a `Model` as so



In [ ]:
# Condition on the observations.
model = epidemic_model(SIRProblem(N), prior_improved)
model_conditioned = model | (in_bed = data.in_bed,)

</div>

<div class="fragment (appear)">

You know what time it is: *inference time*!

</div>



### Metropolis-Hastings (MH)



In [ ]:
chain_mh = sample(model_conditioned, MH(), MCMCThreads(), 10_000, 4; discard_initial=5_000);

Rhat is *okay-ish* but not great, and ESS is pretty low innit?



In [ ]:
plot(chain_mh; size=(800, 500))

Eeehh doesn't look the greatest

Difficult to trust these results, but let's check if it at least did *something* useful



In [ ]:
# We're using the unconditioned model!
predictions_mh = predict(model, chain_mh)

In [ ]:
plot_trajectories!(plot(legend=false, size=(600, 300)), predictions_mh; data=data)

In [ ]:
plot_trajectory_quantiles!(plot(legend=false, size=(600, 300)), predictions_mh; data=data)

Okay, it's not *completely* useless, but my trust-issues are still present.

Metropolis-Hastings have disappointed me one too many times before.



### So instead, let's go `NUTS`



That's right, we're reaching to the **No U-Turn sampler (NUTS)**



#### :PROPERTIES:



[https://chi-feng.github.io/mcmc-demo/app.html](https://chi-feng.github.io/mcmc-demo/app.html)



\*\*



> Wooaah there! `NUTS` requires gradient information!
> 
> How are you going to get that through that `solve`?Good question, voice in my head

I'm obviously not going to it myself



### Automatic differentiation (AD) in Julia



-   [ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jl): forward-mode AD *(default in Turing.jl)*
-   [ReverseDiff.jl](https://github.com/JuliaDiff/ReverseDiff.jl): tape-based reverse-mode AD
-   [Zygote.jl](https://github.com/FluxML/Zygote.jl): source-to-source reverse-mode AD
-   And more&#x2026;

<div class="fragment (appear)">

Up-and-coming

-   [Enzyme.jl](https://github.com/EnzymeAD/Enzyme.jl): Julia bindings for [Enzyme](https://github.com/EnzymeAD/Enzyme.jl) which ADs LLVM (low-level)
-   [Diffractor.jl](https://github.com/JuliaDiff/Diffractor.jl): experimental mixed-mode AD meant to replace Zygote.jl

</div>

<div class="fragment (appear)">

Of importance

-   [ChainRulesCore.jl](https://github.com/JuliaDiff/ChainRulesCore.jl): light-weight package for defining rules, compatible with many of the above

</div>

**Important**

> When you write code, you don't have to make a choice which one you
> want to use!All the (stable) ones, will (mostly) work

*But* how you write code will affect performance characteristics

Takes a bit of know-how + a bit of digging to go properly "vroom!"



### Differentiating through `solve`



With that being said, differentiating through numerical `solve` is not necessarily trivial to do efficiently

There are numerous ways of approaching this problem

![img](.notes/attachments/Bayesian_inference/2023-01-22_12-30-07_Screenshot_20230122_122936.png)

[https://arxiv.org/abs/1812.01892](https://arxiv.org/abs/1812.01892) is *great* resource

<div class="fragment (appear)">

But this is why we have [`SciMLSensitivity.jl`](https://github.com/SciML/SciMLSensitivity.jl)

[SciMLSensitivity.jl docs](https://docs.sciml.ai/SciMLSensitivity/stable/manual/differential_equation_sensitivities/#Choosing-a-Sensitivity-Algorithm) also provides a great overview of different approaches

</div>



In [ ]:
using SciMLSensitivity

It offers

1.  *Discrete sensitivity analysis* or the *"Direct" method*: just use
    `ForwardDiff.Dual` in the `solve`.
2.  *Continuous local sensitivity analysis (CSA)*: extends the original
    system such that the `solve` gives you both the solution and the the
    gradient simultaenously.
3.  *Adjoint methods*: construct a backwards system whose solution gives
    us the gradient.

Just do `solve(problem, solver, sensealg = ...)`



### Back to being `NUTS`



In [ ]:
chain = sample(model_conditioned, NUTS(0.8), MCMCThreads(), 1000, 4);

In [ ]:
chain

Muuuch better! Both ESS and Rhat is looking good



In [ ]:
plot(chain; size=(800, 500))

In [ ]:
# Predict using the results from NUTS.
predictions = predict(model, chain)

In [ ]:
plot_trajectories!(plot(legend=false, size=(600, 300)), predictions; n = 1000, data=data)

In [ ]:
plot_trajectory_quantiles!(plot(legend=false, size=(600, 300)), predictions; data=data)

### Simulation-based calibration (SBC) [Talts et. al. (2018)](https://arxiv.org/abs/1804.06788)



1.  Sample from prior $\theta_1, \dots, \theta_n \sim p(\theta)$.
2.  Sample datasets $\mathcal{D}_i \sim p(\cdot \mid \theta_i)$ for $i = 1, \dots, n$.
3.  Obtain (approximate) $p(\theta \mid \mathcal{D}_i)$ for $i = 1, \dots, n$.

For large enough (n), the "combination" of the posteriors should recover the prior!

"Combination" here usually means computing some statistic and comparing against what it should be

![img](.notes/attachments/Bayesian_inference/2023-01-22_12-09-24_Screenshot_20230122_120848.png)

That's very expensive → in practice we just do this once or twice



In [ ]:
# Sample from the conditioned model so we don't get the `in_bed` variables too
using Random  # Just making usre the numbers of somewhat interesting
rng = MersenneTwister(43);
test_values = rand(rng, NamedTuple, model_conditioned)

Now we condition on those values and run once to generate data



In [ ]:
model_test = model | test_values

In [ ]:
in_best_test = rand(rng, model_test).in_bed;

Next, inference!



In [ ]:
model_test_conditioned = model | (in_bed = in_best_test,)

In [ ]:
# Let's just do a single chain here.
chain_test = sample(model_test_conditioned, NUTS(0.8), 1000);

Did we recover the parameters?

<div class="small-text">



In [ ]:
ps = []
for sym in [:β, :γ, :ϕ⁻¹]
    p = density(chain_test[:, [sym], :])
    vline!([test_values[sym]])
    push!(ps, p)
end
plot(ps..., layout=(3, 1), size=(600, 400))

</div>

Yay!



### Samplers in Turing.jl



-   Metropolis-Hastings, emcee, SGLD ([AdvancedMH.jl](https://github.com/TuringLang/AdvancedMH.jl))
-   Hamiltonian Monte Carlo, NUTS ([AdvancedHMC.jl](https://github.com/TuringLang/AdvancedMH.jl))
-   SMC ([AdvancedPS.jl](https://github.com/TuringLang/AdvancedPS.jl))
-   Elliptical Slice Sampling ([EllipticalSliceSampling.jl](https://github.com/TuringLang/EllipticalSliceSampling.jl))
-   Nested sampling ([NestedSamplers.jl](https://github.com/TuringLang/NestedSamplers.jl))

You can also combine some of these in Turing.jl

<div class="small-text">



In [ ]:
using LinearAlgebra: I

@model function linear_regression(X)
    num_params = size(X, 1)
    β ~ MvNormal(ones(num_params))
    σ² ~ InverseGamma(2, 3)
    y ~ MvNormal(vec(β' * X), σ² * I)
end

# Generate some dummy data.
X = randn(2, 1_000); lin_reg = linear_regression(X); true_vals = rand(lin_reg)

# Condition.
lin_reg_conditioned = lin_reg | (y = true_vals.y,);

</div>

We can then do `Gibbs` but sampling $β$ using `ESS` and $\sigma^2$ using `HMC`



In [ ]:
chain_ess_hmc = sample(lin_reg_conditioned, Gibbs(ESS(:β), HMC(1e-3, 16, :σ²)), 1_000)

Could potentially lead to improvements

**NOTE:** Usually *very* difficult to choose sampler parameters in this case

Means one can also mix discrete and continuous

<div class="small-text">



In [ ]:
@model function mixture(n)
    cluster ~ filldist(Categorical([0.25, 0.75]), n)
    μ ~ MvNormal([-10.0, 10.0], I)
    x ~ arraydist(Normal.(μ[cluster], 1))
end

model_mixture = mixture(10)
fake_values_mixture = rand(model_mixture)
model_mixture_conditioned = model_mixture | (x = fake_values_mixture.x, )
chain_discrete = sample(
    model_mixture_conditioned, Gibbs(PG(10, :cluster), HMC(1e-3, 16, :μ)), MCMCThreads(), 1_000, 4
)

</div>

<div class="x-small-text">



In [ ]:
ps = []
for (i, realizations) in enumerate(eachcol(Array(group(chain_discrete, :cluster))))
    p = density(realizations, legend=false, ticks=false); vline!(p, [fake_values_mixture.cluster[i]])
    push!(ps, p)
end
plot(ps..., layout=(length(ps) ÷ 2, 2), size=(600, 40 * length(ps)))

</div>

Again, this is difficult to get to work properly on non-trivial examples

<u>But</u> it is possible



### Other utilities for Turing.jl



-   [TuringGLM.jl](https://github.com/TuringLang/TuringGLM.jl): GLMs using the formula-syntax from R but using Turing.jl under the hood
-   [TuringBenchmarking.jl](https://github.com/TuringLang/TuringBenchmarking.jl): useful for benchmarking Turing.jl models
-   [TuringCallbacks.jl](https://github.com/TuringLang/TuringCallbacks.jl): on-the-fly visualizations using `tensorboard`

![img](.notes/attachments/Bayesian_inference/2023-01-25_20-50-11_tensorboard_demo_histograms_screen.png)



### Downsides of using Turing.jl



-   Don't do any depedency-extraction of the model ⟹ can't do things like automatic marginalization
    -   *But* it's not impossible; just a matter of development effort
    -   Ongoing work in `TuringLang` to make a [BUGS](https://www.mrc-bsu.cam.ac.uk/software/bugs/) compatible model "compiler" / parser (in colab with Andrew Thomas & others)
-   NUTS performance is at the mercy of AD in Julia
-   You <u>can</u> put anything in your model, but whether you <u>should</u> is a another matter



## Benchmarking



In [ ]:
using SciMLSensitivity
using BenchmarkTools
using TuringBenchmarking

In [ ]:
using ReverseDiff, Zygote

In [ ]:
suite = TuringBenchmarking.make_turing_suite(
    model_conditioned;
    adbackends=[
        TuringBenchmarking.ForwardDiffAD{40,true}(),
        TuringBenchmarking.ReverseDiffAD{false}(),
        TuringBenchmarking.ZygoteAD()
    ]
);
run(suite)

### More data



In [ ]:
# NOTE: We now use 10 000 days instead of just 14.
model_fake = epidemic_model(SIRProblem(N; tspan=(0, 10_000)), prior_improved);

In [ ]:
res = rand(model_fake)
model_fake_conditioned = model_fake | (in_bed = res.in_bed,);

In [ ]:
model_fake_conditioned().infected

In [ ]:
suite = TuringBenchmarking.make_turing_suite(
    model_fake_conditioned;
    adbackends=[
        TuringBenchmarking.ForwardDiffAD{40,true}(),
        TuringBenchmarking.ReverseDiffAD{false}(),
        TuringBenchmarking.ZygoteAD()
    ]
);
run(suite)

## Julia: The Good, the Bad, and the Ugly



An honest take from a little 27-year old Norwegian boy



#### The Good



-   Speed
-   Composability (thank you multiple dispatch)
-   No need to tie yourself to an underlying computational framework
-   Interactive
-   Transparency
-   Very easy to call into other languages



#### Speed



I think you got this already&#x2026;



#### Composability



We've seen some of that

Defining `infected(problem_wrapper, u)` allowed us to abstract away how to extract the compartment of interest



#### Transparency



For starters, almost all the code you'll end up using is pure Julia

Hence, you can always look at the code

You can find the implementation by using `@which`



In [ ]:
# Without arguments
@which sum

In [ ]:
# With arguments
@which sum([1.0])

And yeah, you can even look into the macros

<div class="small-text">



In [ ]:
@macroexpand @model f() = x ~ Normal()

</div>

I told you didn't want to see that.

Can make it *a bit* cleaner by removing linenums:

<div class="x-small-text">



In [ ]:
@macroexpand(@model f() = x ~ Normal()) |> Base.remove_linenums!

</div>



In [ ]:
f(x) = 2x

You can inspect the type-inferred and lowered code



In [ ]:
@code_typed f(1)

You can inspect the LLVM code



In [ ]:
@code_llvm f(1)

And even the resulting machine code



In [ ]:
@code_native f(1)

It really just depends on which level of "I hate my life" you're currently at



#### Calling into other languages



-   [C and Fortran comes built-in stdlib](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/)
-   [RCall.jl](https://juliainterop.github.io/RCall.jl/stable/): call into `R`
-   [PyCall.jl](https://github.com/JuliaPy/PyCall.jl): call into `python`
-   Etc.

When working with `Array`, etc. memory is usually shared ⟹ fairly low overhead



#### C and Fortran



In [ ]:
# Define the Julia function
function mycompare(a, b)::Cint
    println("mycompare($a, $b)")  # NOTE: Let's look at the comparisons made.
    return (a < b) ? -1 : ((a > b) ? +1 : 0)
end

# Get the corresponding C function pointer.
mycompare_c = @cfunction(mycompare, Cint, (Ref{Cdouble}, Ref{Cdouble}))

# Array to sort.
A = [1.3, -2.7, 4.4, 3.1];

# Call in-place quicksort.
ccall(:qsort, Cvoid, (Ptr{Cdouble}, Csize_t, Csize_t, Ptr{Cvoid}),
      A, length(A), sizeof(eltype(A)), mycompare_c)

In [ ]:
# All sorted!
A

[Example is from Julia docs](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/#Creating-C-Compatible-Julia-Function-Pointers)



#### The Bad



Sometimes

-   your code might just slow down without a seemingly good reason,
-   someone did bad, and Julia can't tell which method to call, or
-   someone forces the Julia compiler to compile insane amounts of code



#### "Why is my code suddenly slow?"



One word: **type-instability**

Sometimes the Julia compiler can't quite infer what types fully

<div class="fragment (appear)">

**Result:** python-like performance (for those particular function calls)



In [ ]:
# NOTE: this is NOT `const`, and so it could become some other type
# at any given point without `my_func` knowing about it!
global_variable = 1
my_func_unstable(x) = global_variable * x

In [ ]:
@btime my_func_unstable(2.0);

</div>

Luckily there are tools for inspecting this



In [ ]:
@code_warntype my_func_unstable(2.0)

See that `Any` there? <u>'tis a big no-no!</u>

Once discovered, it can be fixed



In [ ]:
const constant_global_variable = 1
my_func_fixed(x) = constant_global_variable * x
@code_warntype my_func_fixed(2.0)

So long Python performance!



In [ ]:
@btime my_func_fixed(2.0);

*But* this is not always so easy to discover (though this is generally rare)



In [ ]:
# HACK: Here we explicitly tell Julia what type `my_func_unstable`
# returns. This is _very_ rarely a good idea because it just hides
# the underlying problem from `@code_warntype`!
my_func_forced(x) = my_func_unstable(x)::typeof(x)
@code_warntype my_func_forced(2.0)

We can still see the `Any` in there, but on a first glance it looks like `my_func_forced` is type-stable

There are more natural cases where this might occur, e.g. unfortunate closures deep in your callstack

To discovery these there are a couple of more advanced tools:

-   [Cthulhu.jl](https://github.com/JuliaDebug/Cthulhu.jl): Allows you to step through your code like a debugger and perform `@code_warntype`
-   [JET.jl](https://github.com/aviatesk/JET.jl): Experimental package which attempts to automate the process

And even simpler: profile using [ProfileView.jl](https://github.com/timholy/ProfileView.jl) and look for code-paths that *should* be fast but take up a lot of the runtime



In [ ]:
using ProfileView

In [ ]:
@profview foreach(_ -> my_func_unstable(2.0), 1_000_000)

![img](.notes/attachments/Julia:_The_Good,_the_Bad,_and_the_Ugly/2023-01-25_01-16-13_Screenshot_20230125_011603.png)

Note that there's no sign of multiplication here

But most of the runtime is the `./reflection.jl` at the top there

That's Julia looking up the type at runtime



#### Method ambiguity



In [ ]:
ambiguous_function(x, y::Int) = y
ambiguous_function(x::Int, y) = x

# NOTE: Here we have `ambiguous_function(x::Int, y::Int)`
# Which one should we hit?!
ambiguous_function(1, 2)

But here Julia warns us, and so we can fix this by just doing as it says: define `ambiguous_function(::Int64, ::Int64)`



In [ ]:
ambiguous_function(::Int64, ::Int64) = "neato"
ambiguous_function(1, 2)

#### Long compilation times



In Julia, for better or worse, we can generate code

**Problem:** it can be *lots* of code of we really want to

**Result:** first execution can be *slow*

<div class="fragment (appear)">

**Time to first plot (TTFP)** is Julia's worst enemy

But things are always improving

![img](.notes/attachments/Julia:_The_Good,_the_Bad,_and_the_Ugly/2023-01-25_01-29-05_Screenshot_20230125_012853.png)

</div>



#### Another example: mis-use of `@generated`



In [ ]:
# NOTE: `@generated` only has access to static information, e.g. types of arguments.
# Here I'm using the special type `Val` to make a number `N` static.
@generated function unrolled_addition(::Val{N}) where {N}
    expr = Expr(:block)
    push!(expr.args, :(x = 0))
    for i = 1:N
        push!(expr.args, :(x += $(3.14 * i)))
    end

    return expr
end

When I call this with some `Val(N)`, Julia will execute this *at compile-time*!



In [ ]:
# NOTE: At runtime, it then just returns the result immediately
@code_typed unrolled_addition(Val(10))

But if I just change the value `10` to `11`, it's a *completely* different type!

So Julia has to compile `unrolled_addition` from scratch



In [ ]:
@time @eval unrolled_addition(Val(11));

Or a bit crazier



In [ ]:
@time @eval unrolled_addition(Val(10_001));

Here it took ~0.4s, of which 99.95% was compilation time

I think you get the idea

But boy is it fast to run!



In [ ]:
@btime unrolled_addition(Val(10_001));

In [ ]:
function not_unrolled_addition(N)
    x = 0
    for i = 1:N
        x += 3.14 * i
    end

    return x
end

In [ ]:
@btime not_unrolled_addition(10_001);

**Funny side-note:** at first I did the following



In [ ]:
@generated function unrolled_addition_old(::Val{N}) where {N}
    expr = Expr(:block)
    push!(expr.args, :(x = 0))
    for i = 1:N
        push!(expr.args, :(x += $i))  # NOTE: No 3.14!
    end
    return expr
end
function not_unrolled_addition_old(N)
    x = 0
    for i = 1:N
        x += i  # NOTE: No 3.14!
    end
    return x
end

In [ ]:
@btime unrolled_addition_old(Val(10_001));
@btime not_unrolled_addition_old(10_001);

LLVM probably recognized the pattern of `not_unrolled_addition_old` and unrolls it for us

Let's check!



In [ ]:
# NOTE: The one LLVM failed to unroll
@code_llvm not_unrolled_addition(10_001)

In [ ]:
# NOTE: The one LLVM seems to have unrolled.
@code_llvm not_unrolled_addition_old(10_001)

#### The Ugly



<u>**Reverse-mode automatic differentiation**</u>

ForwardDiff.jl is a pure joy, but slows down as dimensionality grows

Then one should reach for ReverseDiff.jl or Zygote.jl

<div class="fragment (appear)">

Most of the time it works really well, but sometimes you hit a real sharp edge

And sharp edges cut; they cut *deep*

Like <u>"16X slower when the function is implemented more efficiently"-deep</u>

![img](.notes/attachments/Julia:_The_Good,_the_Bad,_and_the_Ugly/2023-01-25_01-01-31_Screenshot_20230125_010111.png)

</div>

<div class="fragment (appear)">

If you want to see a man in pain, you can find the full issue [here](https://github.com/TuringLang/Turing.jl/issues/1934)

On the flip-side, once addressed (a type-instability), it's [3X faster than before](https://github.com/TuringLang/DistributionsAD.jl/pull/231)

</div>



#### Overall



Julia is pretty darn awesome

Easy to get going, and you can always make it faster by just optimizing your Julia code

No need to drop down to C++

Buuuut it can't beat Python at deep learning

Otherwise, it's worth a try

Godspeed to you

Fin.

